# Elevator FSM (Mealy) with `transitions`

This notebook implements an elevator controller finite-state machine (FSM) using the `transitions` library.
- States: IDLE, MOVE_UP, MOVE_DOWN, DOOR_OPENING, DOOR_OPEN, DOOR_CLOSING, FAULT
- Inputs (events): call, arrived, door_opened, timeout, door_closed, obstacle, emergency, reset
- Outputs (Mealy): depend on (state, event), e.g. lift_motor_up_on, display_fault, ...

## Setup transitions and constants


In [ ]:
!pip -q install transitions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.8/112.8 kB 5.0 MB/s eta 0:00:00


In [ ]:
from transitions import Machine

## Create ElevatorFSM and init Machine

The elevator is used to move between 3 floors of the building and handle only 1 requirement at a time.

In [72]:
# Function helper: Print output messages (from output signals)
def emit(*signals):
    for s in signals:
        if s == "none":
            continue
        print(OUTPUT_TEXT.get(s, s))

OUTPUT_TEXT = {
    "door_motor_open": "Door motor is opening...",
    "door_motor_close": "Door motor is closing...",
    "lift_motor_up_on": "Lift motor is moving up...",
    "lift_motor_down_on": "Lift motor is moving down...",
    "lift_motor_stop": "Lift motor is stopping...",
    "alarm_on": "Alarm is ON now...",
    "alarm_off": "Alarm is OFF now...",
    "display_fault": "Display is showing FAULT...",
    "display_idle": "Display is showing IDLE...",
    "display_arrived": "Display is showing ARRIVED...",
    "display_door_open": "Display is showing DOOR OPEN...",
    "display_closing": "Display is showing CLOSING...",
    "display_moving_up": "Display is showing MOVING UP...",
    "display_moving_down": "Display is showing MOVING DOWN...",
}

In [ ]:
class ElevatorFSM:
    def __init__(self, start_floor: int = 1):
        self.current_floor = start_floor
        self.target_floor = None

    def set_target(self, floor: int):
        self.target_floor = floor

    def clear_target(self, event):
        self.target_floor = None

    # === OUTPUT ACTIONS ===
    # --- Lift motor outputs ---
    def out_lift_up(self, event):
        emit("lift_motor_up_on", "display_moving_up")

    def out_lift_down(self, event):
        emit("lift_motor_down_on", "display_moving_down")

    def out_lift_stop_fault(self, event):
        emit("lift_motor_stop", "alarm_on", "display_fault")

    # --- Door outputs ---
    def out_open_door_arrived(self, event):
        emit("lift_motor_stop", "door_motor_open", "display_arrived")

    def out_open_door_same_floor(self, event):
        emit("door_motor_open", "display_arrived")

    def out_display_door_open(self, event):
        emit("display_door_open")

    def out_close_door(self, event):
        emit("door_motor_close", "display_closing")

    def out_display_idle(self, event):
        emit("display_idle")

    def out_obstacle_reopen(self, event):
        emit("door_motor_open", "display_door_open")

    # --- Fault outputs ---
    def out_fault_no_lift_stop(self, event):
        emit("alarm_on", "display_fault")

    def out_reset_from_fault(self, event):
        emit("alarm_off", "display_idle")

Guard conditions for call events

In [ ]:
class ElevatorFSM(ElevatorFSM):
    def cond_call_up(self, event):
        floor = event.kwargs["floor"]
        return floor > self.current_floor

    def cond_call_down(self, event):
        floor = event.kwargs["floor"]
        return floor < self.current_floor

    def cond_call_same(self, event):
        floor = event.kwargs["floor"]
        return floor == self.current_floor

Before and after moving

In [ ]:
class ElevatorFSM(ElevatorFSM):
    def before_set_target(self, event):
        floor = event.kwargs["floor"]
        self.set_target(floor)

    def after_arrived_update_floor(self, event):
        # when arrived, elevator reaches target_floor
        if self.target_floor is not None:
            self.current_floor = self.target_floor

Init states and transitions

In [ ]:
STATES = [
    "IDLE",
    "MOVE_UP",
    "MOVE_DOWN",
    "DOOR_OPENING",
    "DOOR_OPEN",
    "DOOR_CLOSING",
    "FAULT",
]

transitions_list = [
    # --- IDLE: call(floor) split into 3 via conditions ---
    {
        "trigger": "call",
        "source": "IDLE",
        "dest": "MOVE_UP",
        "conditions": "cond_call_up",
        "before": "before_set_target",
        "after": "out_lift_up",
    },
    {
        "trigger": "call",
        "source": "IDLE",
        "dest": "MOVE_DOWN",
        "conditions": "cond_call_down",
        "before": "before_set_target",
        "after": "out_lift_down",
    },
    {
        "trigger": "call",
        "source": "IDLE",
        "dest": "DOOR_OPENING",
        "conditions": "cond_call_same",
        "before": "before_set_target",
        "after": "out_open_door_same_floor",
    },

    # --- Emergency (per lambda table) ---
    {
        "trigger": "emergency",
        "source": ["IDLE", "MOVE_UP", "MOVE_DOWN"],
        "dest": "FAULT",
        "after": "out_lift_stop_fault",
    },
    {
        "trigger": "emergency",
        "source": ["DOOR_OPENING", "DOOR_OPEN", "DOOR_CLOSING"],
        "dest": "FAULT",
        "after": "out_fault_no_lift_stop",
    },

    # --- Moving: arrived -> DOOR_OPENING ---
    {
        "trigger": "arrived",
        "source": ["MOVE_UP", "MOVE_DOWN"],
        "dest": "DOOR_OPENING",
        "after": ["after_arrived_update_floor", "out_open_door_arrived"],
    },

    # --- Door opening complete ---
    {
        "trigger": "door_opened",
        "source": "DOOR_OPENING",
        "dest": "DOOR_OPEN",
        "after": "out_display_door_open",
    },

    # --- Door open timeout -> closing ---
    {
        "trigger": "timeout",
        "source": "DOOR_OPEN",
        "dest": "DOOR_CLOSING",
        "after": "out_close_door",
    },

    # --- Obstacle during closing -> opening ---
    {
        "trigger": "obstacle",
        "source": "DOOR_CLOSING",
        "dest": "DOOR_OPENING",
        "after": "out_obstacle_reopen",
    },

    # --- Door closed -> IDLE (clear target) ---
    {
        "trigger": "door_closed",
        "source": "DOOR_CLOSING",
        "dest": "IDLE",
        "after": ["clear_target", "out_display_idle"],
    },

    # --- Reset from FAULT -> IDLE ---
    {
        "trigger": "reset",
        "source": "FAULT",
        "dest": "IDLE",
        "after": ["clear_target", "out_reset_from_fault"],
    },
]

In [ ]:
e = ElevatorFSM(start_floor=1)

machine = Machine(
    model=e,
    states=STATES,
    initial="IDLE",
    auto_transitions=False,
    send_event=True,
    ignore_invalid_triggers=True,
)

machine.add_transitions(transitions_list)

## Check scenario

In [69]:
# function helpers
def run_scenario(title, fn):
    print("\n" + "="*60)
    print(title)
    print("="*60)
    fn()

def state_now(elevator):
  print("Elevator now is on the floor:" , elevator.current_floor, "with state:", elevator.state, ".\n")

### Scenario 1: Call same floor

In [81]:
def scenario_same_floor():
    e.current_floor = 2
    e.clear_target(None)
    machine.set_state("IDLE")

    state_now(e) # init state

    print("Customers called the elevator to the 2rd floor.")
    e.call(floor=2)   # call(target floor = current floor)
    state_now(e)

run_scenario("Scenario 1: Call the same floor -> immediately open door", scenario_same_floor)


Scenario 1: Call the same floor -> immediately open door
Elevator now is on the floor: 2 with state: IDLE .

Customers called the elevator to the 2rd floor.
Door motor is opening...
Display is showing ARRIVED...
Elevator now is on the floor: 2 with state: DOOR_OPENING .



### Scenario 2: Obstacle while closing door

In [84]:
def scenario_obstacle():
    e.current_floor = 1
    e.clear_target(None)
    machine.set_state("IDLE")

    state_now(e) # init state

    print("Customers called the elevator to the 2nd floor.")
    e.call(floor=2)
    state_now(e)

    print("The elevator arrived.")
    e.arrived()
    state_now(e)

    print("The door has opened.")
    e.door_opened()
    state_now(e)

    print("Wait time is over. The elevator began to close.")
    e.timeout()
    state_now(e)

    print("Customers put their feet to block the elevator door.")
    e.obstacle()
    state_now(e)

    print("The door opened again.")
    e.door_opened()
    state_now(e)

run_scenario("Scenario 2: Obstacle exists during closing -> reopen door", scenario_obstacle)


Scenario 2: Obstacle exists during closing -> reopen door
Elevator now is on the floor: 1 with state: IDLE .

Customers called the elevator to the 2nd floor.
Lift motor is moving up...
Display is showing MOVING UP...
Elevator now is on the floor: 1 with state: MOVE_UP .

The elevator arrived.
Lift motor is stopping...
Door motor is opening...
Display is showing ARRIVED...
Elevator now is on the floor: 2 with state: DOOR_OPENING .

The door has opened.
Display is showing DOOR OPEN...
Elevator now is on the floor: 2 with state: DOOR_OPEN .

Wait time is over. The elevator began to close.
Door motor is closing...
Display is showing CLOSING...
Elevator now is on the floor: 2 with state: DOOR_CLOSING .

Customers put their feet to block the elevator door.
Door motor is opening...
Display is showing DOOR OPEN...
Elevator now is on the floor: 2 with state: DOOR_OPENING .

The door opened again.
Display is showing DOOR OPEN...
Elevator now is on the floor: 2 with state: DOOR_OPEN .



### Scenario 3: Emergency + reset

In [85]:
def scenario_emergency_reset():
    e.current_floor = 1
    e.clear_target(None)
    machine.set_state("IDLE")

    state_now(e)  # init state

    print("Customers called the elevator to the 3rd floor.")
    e.call(floor=3)
    state_now(e)

    print("Emergency button was pressed. The elevator entered FAULT mode.")
    e.emergency()
    state_now(e)

    print("Technician reset the system. The elevator returned to IDLE mode.")
    e.reset()
    state_now(e)

run_scenario("Scenario 3: emergency -> FAULT -> reset -> IDLE", scenario_emergency_reset)


Scenario 3: emergency -> FAULT -> reset -> IDLE
Elevator now is on the floor: 1 with state: IDLE .

Customers called the elevator to the 3rd floor.
Lift motor is moving up...
Display is showing MOVING UP...
Elevator now is on the floor: 1 with state: MOVE_UP .

Emergency button was pressed. The elevator entered FAULT mode.
Lift motor is stopping...
Alarm is ON now...
Display is showing FAULT...
Elevator now is on the floor: 1 with state: FAULT .

Technician reset the system. The elevator returned to IDLE mode.
Alarm is OFF now...
Display is showing IDLE...
Elevator now is on the floor: 1 with state: IDLE .



### Scenario 4: Call while moving (will be ignored)

In [86]:
def scenario_call_while_moving():
    e.current_floor = 1
    e.clear_target(None)
    machine.set_state("IDLE")

    state_now(e)  # init state

    print("Customers called the elevator to the 3rd floor.")
    e.call(floor=3)  # IDLE -> MOVE_UP
    state_now(e)

    print("While the elevator was moving up, another customer called it to the 2nd floor.")
    print("This request should be ignored because the elevator processes only one request at a time.")
    e.call(floor=2)  # ignored (no transition from MOVE_UP for call)
    state_now(e)

run_scenario("Scenario 4: call while moving is ignored", scenario_call_while_moving)


Scenario 4: call while moving is ignored
Elevator now is on the floor: 1 with state: IDLE .

Customers called the elevator to the 3rd floor.
Lift motor is moving up...
Display is showing MOVING UP...
Elevator now is on the floor: 1 with state: MOVE_UP .

While the elevator was moving up, another customer called it to the 2nd floor.
This request should be ignored because the elevator processes only one request at a time.
Elevator now is on the floor: 1 with state: MOVE_UP .

